# Exploring and Visualizing CMI Sleep Dataset 
In this notebook I perform an exploratory analaysis and visualization of the Child Mind Institute sleep monitoring dataset, as part of the [Detect Sleep States](https://www.kaggle.com/competitions/child-mind-institute-detect-sleep-states/overview) competition.

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import polars as pl
import datetime 
from tqdm import tqdm

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

## Importing data
Since the time series dataset is very large, we will use [`polars`](https://www.pola.rs) in this notebook as opposed to the more conventional `pandas` library. We begin by importing the datasets and extracting the time data from the 'timestamp' field. Wherever possible we will cast our columns into efficient datatypes to save on memory.

In [ ]:
# Importing data 
dt_transforms = [
    pl.col('timestamp').str.to_datetime(), 
    (pl.col('timestamp').str.to_datetime().dt.year()-2000).cast(pl.UInt8).alias('year'), 
    pl.col('timestamp').str.to_datetime().dt.month().cast(pl.UInt8).alias('month'),
    pl.col('timestamp').str.to_datetime().dt.day().cast(pl.UInt8).alias('day'), 
    pl.col('timestamp').str.to_datetime().dt.hour().cast(pl.UInt8).alias('hour')
]

data_transforms = [
    pl.col('anglez').cast(pl.Int8), # Casting anglez to 8 bit integer
    (pl.col('enmo')*1000).cast(pl.UInt16), # Convert enmo to 16 bit uint
]

train_series = pl.scan_parquet('/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet').with_columns(
    dt_transforms + data_transforms
    )

train_events = pl.read_csv('/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv').with_columns(
    dt_transforms
    )

test_series = pl.scan_parquet('/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet').with_columns(
    dt_transforms + data_transforms
    )

# Getting series ids as a list for convenience
series_ids = train_events['series_id'].unique(maintain_order=True).to_list()

## Visualizing time series
Let's create a function to visualize given series in a way which displays our two main datapoints, `anglez` and `enmo`, along with the onset and wakeup events which we would like to predict - or equivalently, combining the two, the sleeping periods vs. the wakeful periods of the time series. Our visualization should also be able to handle missing data by marking days with missing values.

To begin, let's find out whether the number of onset events always equals the number of wakeup events. In principle these numbers should always be the same.

In [ ]:
onset_counts = train_events.filter(pl.col('event')=='onset').group_by('series_id').count().sort('series_id')['count']
wakeup_counts = train_events.filter(pl.col('event')=='wakeup').group_by('series_id').count().sort('series_id')['count']

counts = pl.DataFrame({'series_id':sorted(series_ids), 'onset_counts':onset_counts, 'wakeup_counts':wakeup_counts})
count_mismatches = counts.filter(counts['onset_counts'] != counts['wakeup_counts'])

print(f'Series where the number of onsets is not the same as the number of wakeups: {count_mismatches.shape[0]}')

So there are four series where there is a mismatch. Let's remove these from our training set.

In [ ]:
train_series = train_series.filter(~pl.col('series_id').is_in(count_mismatches['series_id']))
train_events = train_events.filter(~pl.col('series_id').is_in(count_mismatches['series_id']))

# Updating series_ids list
series_ids = train_events.drop_nulls()['series_id'].unique(maintain_order=True).to_list()

We also note the presence of missing labels in some of the series. That is, for some series there are days in which no onsets or wakeups are recorded, even though the time series data indicate periods where the participant likely was sleeping. Let's see how frequently this appears in our data. 

In [ ]:
missing_data = train_events.group_by('series_id', maintain_order=True).agg(pl.col('step').map_elements(lambda x: x.is_null().any()).alias('MissingValues'))
print(f"Series with missing data: {missing_data['MissingValues'].sum()}/{train_events['series_id'].n_unique()}")

Unfortunately a large majority of series contain days with missing data. We will take this into consideration when training our model.

Now that we can be certain that the onsets and wakeup numbers match, we can define a plotting function that will display our two primary datapoints - anglez and enmo - on a time series for a given series_id, with the sleep stages highlighted. For completeness, we will also flag any days where there are missing or null values. 

In [ ]:
def getdata(i, every=1) : 
    '''
    Get data for a given sample series. 
    '''
    
    idx = series_ids[i] if type(i) == int else i
    series = train_series.filter((pl.col('series_id') == idx) & (pl.col('step') % every == 0)).collect()
    events = train_events.filter(pl.col('series_id') == idx)
    
    return idx, series, events

def plotdata(idx, series, events) :
    '''
    Plots a time series of anglez and enmo for the i-th series, using a 
    given frequency of samples, with wakeup and onset events marked. 
    every=1 uses every sample at 5 minute intervals, and every=12 gives 
    minute-to-minute time series. 
    '''
    
    fig = make_subplots(specs=[[{'secondary_y': True}]])
    
    # Plotting time series data
    fig.add_trace(go.Scatter(x=series['timestamp'], y=series['anglez'], mode='lines', name='anglez'), secondary_y=False)
    fig.add_trace(go.Scatter(x=series['timestamp'], y=series['enmo'], mode='lines', name='enmo'), secondary_y=True)
    
    # Plotting sleeping periods
    onsets = events.filter((pl.col('event') == 'onset') & (pl.col('timestamp') != None))['timestamp'].to_list()
    wakeups = events.filter((pl.col('event') == 'wakeup') & (pl.col('timestamp') != None))['timestamp'].to_list()
    for onset, wakeup in zip(onsets, wakeups) :
        fig.add_vrect(
            x0=onset, x1=wakeup,
            fillcolor='gray', opacity=0.2,
            layer='below', line_width=0
        )
        
    # Add dummy trace for the legend
    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='lines',
        line=dict(color='gray', width=0),
        fill='tozeroy',
        fillcolor='gray',
        opacity=0.2,
        name="Sleeping periods"
    ))
        
    # Plotting days with missing data
    start_date = events['timestamp'].min().replace(hour=0, minute=0, second=0, microsecond=0)
    null_nights = events.filter(pl.col('step').is_null())['night'].unique().to_list()
    for n in null_nights : 
        fig.add_vrect(
            x0=start_date + datetime.timedelta(days=n-1), x1=start_date + datetime.timedelta(days=n),
            fillcolor='violet', opacity=0.2,
            layer='below', line_width=0
        )
    
    # Add dummy trace for the legend
    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='lines',
        line=dict(color='violet', width=0),
        fill='tozeroy',
        fillcolor='violet',
        opacity=0.2,
        name="Missing data"
    ))
        
    fig.update_layout(
        title_text=f"Accelerometer Data<br><sub>ID: {idx}</sub>", 
        #template = 'ggplot2',
        xaxis=dict(showgrid=False), yaxis=dict(showgrid=False), yaxis2=dict(showgrid=False)
    )
    
    fig.show()
    
# data = getdata(10)

# plotdata(*data)

After plotting a few series samples, we can see some very clear patterns that tend to distinguish sleeping periods from wakeful periods. 
- During wakeful periods `anglez` varies constantly, rarely ever staying at the same value for consecutive measurements. During sleeping periods there tend to be periods of low to zero variability in `anglez` where the person will change positions and then remain there for many consecutive measurements. 
- During wakeful periods `enmo` also has much higher variability and reaches much greater values, while during sleeping periods `enmo` values stay very close to 0.
- Days with missing data tend to have much smaller variations in `anglez`, as well as `enmo`. 



## Feature Engineering

Let's start by looking at the series without any null values to look for features correlated with sleep/wakefullness, without being impacted by missing data.

In [ ]:
i = 0

full_data_ids = missing_data.filter(pl.col('MissingValues')==False)['series_id'].to_list()
idx, series, events = getdata(full_data_ids[i])
plotdata(idx, series, events)

onsets = events.filter((pl.col('event') == 'onset') & (pl.col('timestamp') != None))['timestamp'].to_list()
wakeups = events.filter((pl.col('event') == 'wakeup') & (pl.col('timestamp') != None))['timestamp'].to_list()

Let's put together some features to see if we can find any that strongly correlate with sleeping phases. 

In [ ]:
features = [sum([(onset <= pl.col('timestamp')) & (pl.col('timestamp') <= wakeup) for onset, wakeup in zip(onsets, wakeups)]).alias('asleep')]
feature_cols = ['asleep']

rolling_durs = [1, 30, 60*3] # Rolling aggregate durations, in minutes

for var in ['anglez', 'enmo'] :
    features += [pl.col(var).alias(var), 
                 pl.col(var).diff().fill_null(0).abs().alias(f'{var}_1v')
                ]
    feature_cols += [var, f'{var}_1v']
    
    # Rolling aggregates
    for dur in rolling_durs :
        features += [
            pl.col(var).rolling_mean(dur * 12, center=True, min_periods=1).alias(f'{var}_{dur}m_mean'), # Rolling mean
           
            
            # Rolling total variations
            pl.col(var).diff().fill_null(0).abs().rolling_mean(dur * 12, center=True, min_periods=1).alias(f'{var}_1v_{dur}m_mean'),
            
        ]

        feature_cols += [
            f'{var}_{dur}m_mean', f'{var}_1v_{dur}m_mean'
        ]
        
# Lets also include enmo x anglez features
features += [
    (pl.col('anglez') * pl.col('enmo')).alias('anglez.enmo'), 
    (pl.col('anglez') * pl.col('enmo')).alias('anglez.enmo').diff().fill_null(0).abs().alias('anglez.enmo_1v')
]

feature_cols += [
    'anglez.enmo', 'anglez.enmo_1v'
]

for dur in rolling_durs :
    features += [
        (pl.col('anglez') * pl.col('enmo')).rolling_mean(dur * 12, center=True, min_periods=1).alias(f'anglez.enmo_{dur}m_mean'), # Rolling mean
        

        # Rolling total variations
        (pl.col('anglez') * pl.col('enmo')).diff().fill_null(0).abs().rolling_mean(dur * 12, center=True, min_periods=1).alias(f'anglez.enmo_1v_{dur}m_mean'),
        
    ]

    feature_cols += [
        f'anglez.enmo_{dur}m_mean', f'anglez.enmo_1v_{dur}m_mean'
    ]

corr_df = series.with_columns(features)[feature_cols].corr()

In [ ]:
df = series.with_columns(features)

In [ ]:
print(r'Z-Angle \times ENMO')

In [ ]:
from plotly.subplots import make_subplots

N = (len(feature_cols)-1)//3

#fig = make_subplots(cols=1, rows=3)
variables = ['Z-Angle', 'ENMO']
for c in range(2) :
    fig = go.Figure()
    cols = feature_cols[:1] + feature_cols[c*N + 1:(c+1)*N + 1]
    fig.add_trace(
        go.Heatmap(
            x = cols,
            y = cols,
            z = df[cols].corr(),
            text=df[cols].corr(),
            texttemplate='%{text:.2f}',
            zmin=-1,zmax=1
        )
    )
    fig.update_layout(title_text=f"{variables[c]} correlation map<br><sub>ID: {idx}<sub>")
    fig.show()

In [ ]:
# features = [sum([(onset <= pl.col('timestamp')) & (pl.col('timestamp') <= wakeup) for onset, wakeup in zip(onsets, wakeups)]).alias('asleep')]
# feature_cols = ['asleep']

# rolling_durs = [1, 30, 60*3] # Rolling aggregate durations, in minutes

# for var in ['anglez', 'enmo'] :
#     features += [pl.col(var).alias(var), pl.col(var).diff().abs().alias(f'{var}_1v')]
#     feature_cols += [var, f'{var}_1v']
    
#     # Rolling aggregates
#     for dur in rolling_durs :
#         features += [
#             pl.col(var).rolling_mean(dur * 12, center=True, min_periods=1).alias(f'{var}_{dur}m_mean'), # Rolling mean
           
#             # Rolling total variations
#             pl.col(var).diff().abs().rolling_mean(dur * 12, center=True, min_periods=1).alias(f'{var}_1v_{dur}m_mean'),
#                 ]

#         feature_cols += [
#             f'{var}_{dur}m_mean', f'{var}_1v_{dur}m_mean'
#         ]
        
# # Lets also include enmo x anglez features
# features += [
#     (pl.col('anglez') * pl.col('enmo')).alias('anglez.enmo'), 
#     (pl.col('anglez') * pl.col('enmo')).alias('anglez.enmo').diff().abs().alias('anglez.enmo_1v')
# ]

# feature_cols += [
#     'anglez.enmo', 'anglez.enmo_1v'
# ]

# for dur in rolling_durs :
#     features += [
#         (pl.col('anglez') * pl.col('enmo')).rolling_mean(dur * 12, center=True, min_periods=1).alias(f'anglez.enmo_{dur}m_mean'), # Rolling mean
       
#         # Rolling total variations
#         (pl.col('anglez') * pl.col('enmo')).diff().abs().rolling_mean(dur * 12, center=True, min_periods=1).alias(f'anglez.enmo_1v_{dur}m_mean')
#           ]

#     feature_cols += [
#         f'anglez.enmo_{dur}m_mean', f'anglez.enmo_1v_{dur}m_mean'
#     ]

# corr_df = series.with_columns(features)


# fig = go.Figure()
# fig.add_trace(
#     go.Heatmap(
#         x = cols,
#         y = cols,
#         z = corr_df[cols].drop_nulls().corr(),
#         text=corr_df[cols].drop_nulls().corr(),
#         texttemplate='%{text:.2f}'
#     )
# )
# fig.update_layout(title_text=f"Feature correlation map<br><sub>ID: {idx}<sub>")
# fig.show()

Some observations: 
- The longer 30 minute rolling windows outperformed both the minute rolling window and the individual datapoints by a wide margin.
- The feature correlated most closely with sleeping is the 30 minute rolling average of the first order variation of anglez, with a 75% negative correlation.
- Both the enmo 30 minute rolling average and the enmo 30 minute rolling first order variation offered strong negative correlations. 
- The rolling 30 minute multiplicative combitionation of anglez and enmo offered a strong positive correlation as well. 

Let's see how these four correlations hold up across the other sample series with full data.

In [ ]:
cols = ['asleep', 'anglez_1v_r30', 'enmo_1v_r30', 'enmo_r30', 'anglezxenmo_r30', 'enmo_1v_30m_mean', 'anglez_1v_30m_mean']
sleep_corr = pl.DataFrame()

for idx in tqdm(full_data_ids) : 
    idx, series, events = getdata(idx)

    onsets = events.filter((pl.col('event') == 'onset') & (pl.col('timestamp') != None))['timestamp'].to_list()
    wakeups = events.filter((pl.col('event') == 'wakeup') & (pl.col('timestamp') != None))['timestamp'].to_list()
    
    features = series.with_columns(
        sum([(onset <= pl.col('timestamp')) & (pl.col('timestamp') <= wakeup) for onset, wakeup in zip(onsets, wakeups)]).alias('asleep'), 
        pl.col('enmo').rolling_mean(12*30, center=True).alias('enmo_r30'), # Rolling mean of enmo for 30 minutes
        (pl.col('enmo') * pl.col('anglez')).rolling_mean(12*30, center=True).alias('anglezxenmo_r30'), # Rolling mean of anglez times enmo for 30 minutes
        (pl.col('anglez').shift(-1) - pl.col('anglez')).abs().rolling_mean(12*30, center=True).alias('anglez_1v_r30'), # 1st order variation of anglez for 30 min
        (pl.col('enmo').shift(-1) - pl.col('enmo')).abs().rolling_mean(12*30, center=True).alias('enmo_1v_r30') # First order variation of enmo for 30 minutes
    )
    
    # Calculate 'enmo_1v' and 'anglez_1v' columns before using them
    features = features.with_columns(
        pl.col('enmo').diff().abs().alias('enmo_1v'),
        pl.col('anglez').diff().abs().alias('anglez_1v')
    )
    
    for dur in [30]:  # You can add other durations if needed
        features = features.with_columns(
            pl.col('enmo_1v').rolling_mean(dur * 12, center=True, min_periods=1).alias(f'enmo_1v_{dur}m_mean'),
            pl.col('anglez_1v').rolling_mean(dur * 12, center=True, min_periods=1).alias(f'anglez_1v_{dur}m_mean')
        )
    
    corr_df = features
    
    sleep_corr = sleep_corr.vstack(corr_df[cols].drop_nulls().corr()[0].drop('asleep').with_columns(pl.lit(idx).alias('ID')))


In [ ]:
plot_cols = [col for col in cols if col != 'asleep']

fig = go.Figure()
for col in plot_cols : 
    fig.add_trace(go.Violin(y=sleep_corr[col],
                            name=col,
                            box_visible=True,
                            meanline_visible=True))
    
fig.update_layout(title_text='Feature correlation with sleep across all full-data time series')
    
fig.show()

Our rolling average anglez $\times$ enmo feature's correlation with sleep does not appear to be robust across the other time series, so we will discard it and move forward with the remaining three features, whose correlations do appear to be robust. 

### Exploiting periodicity
NOTE: I'll come back to this later. 

In [ ]:
px.histogram(train_events, x='hour', color='event', opacity=0.5, barmode='overlay')

## Training Random Forest Model
This section has been moved to my [Feature Engineering and Random Forest Prediction](https://www.kaggle.com/code/lccburk/feature-engineering-and-random-forest-prediction) notebook.